# Coloring

## Prerequisites
Import the required modules and run a few models with a small orbit library.

In [ ]:
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import cmasher
import pymc as pm
from vorbin.voronoi_2d_binning import voronoi_2d_binning

import dynamite as dyn

In [ ]:
print('DYNAMITE version', dyn.__version__)
print('    installed at ', dyn.__path__)  # Uncomment to print the complete DYNAMITE installation path

fname = 'FCC167/FCC167.yaml'
c = dyn.config_reader.Configuration(fname, reset_logging=True)
# c = dyn.config_reader.Configuration(fname, reset_logging=True, reset_existing_output=True)
# _ = dyn.model_iterator.ModelIterator(c)

## Voronoi orbit binning
Before binning the orbits, let's have a look at the $r$ - $\lambda_z$ phase space.

In [ ]:
# Number of desired r and lambda_z bins
nr = 6
nl = 7

In [ ]:
plotter = dyn.plotter.Plotter(c)

In [ ]:
# # NOTE: when using force_lambda_z=True, then the titles of the orbit-distribution plots are incorrect: all orbits are shown in this distribution - not only short-axis tubes!
# fig1 = plotter.orbit_distribution(model=None, minr=None, maxr=None, r_scale='log', nr=nr, nl=nl,
#                                   orientation='vertical', subset='short', force_lambda_z=True)
# fig2 = plotter.orbit_distribution(model=None, minr=None, maxr=None, r_scale='linear', nr=nr, nl=nl,
#                                   orientation='vertical', subset='short', force_lambda_z=True)

Perform Voronoi binning of orbits in the radius-circularity phase space, based on the best-fitting model so far. The goal is to group the original
``n_orbits`` orbit bundles into fewer ``n_bundle`` Voronoi bundles with each of these Voronoi bundles accounting for
a weight of at least ``vor_weight``.

Note that each original orbit bundle is just one orbit if ``dithering=1``, but an actual bundle if ``dithering > 1``.

The result is a tuple ``(vor_bundle_mapping, phase_space_binning)``:
```
vor_bundle_mapping :    np.array of shape (n_bundle, n_orbits)
                        Mapping between the "original" orbit bundles and the Voronoi
                        orbit bundles: vor_bundle_mapping(i_bundle, i_orbit) is the
                        fraction of i_orbit assigned to i_bundle, multiplied by i_orbit's weight.
phase_space_binning :   dict
                        'in':   np.array of shape (3, nr*nl), the binning input:
                                bin r, bin lambda_z, bin total weight
                        'out':  np.array of shape (3, n_bundle), the Voronoi binning output:
                                weighted Voronoi bin centroid coordinates r_bar, lambda_bar
                                and Voronoi bin total weights
                        'map':  np.array of shape (nr*nl,) the phase space mapping:
                                Voronoi bin numbers for each input bin
```

In [ ]:
coloring = dyn.coloring.Coloring(c)  # the phase space binning resides in the Coloring class (might move to Analysis)
vor_weight = 0.05  # define the desired (minimum) total orbital weight in each Voronoi bin
vor_bundle_mapping, phase_space_binning = coloring.bin_phase_space(model=None,
                                                                   minr='auto',
                                                                   maxr='auto',
                                                                   r_scale='linear',
                                                                   nr=nr,
                                                                   nl=nl,
                                                                   vor_weight=vor_weight,
                                                                   vor_ignore_zeros=False,
                                                                   make_diagnostic_plots=True,
                                                                   extra_diagnostic_output=True,
                                                                   cvt=False,
                                                                   wvt=False)

Let's see how much weight each orbit contributes to each orbit bundle. If ``dithering=1``, then each orbit will contribute its weight to exactly one Voronoi bundle. If ``dithering > 1``, then each orbit actually represents a bundle by itself and can hence contribute to multiple (neighboring) Voronoi orbit bundles.

In [ ]:
# If dithering > 1, orbit bundles may contribute to multiple Voronoi orbit bundles
plt.figure(figsize=(24,4))
plt.gca().set_title('Weight that each orbit bundle contributes to Voronoi orbit bundles')
plt.pcolormesh(np.log10(vor_bundle_mapping), shading='flat', cmap='Greys')
plt.xlabel('Orbit bundle id')
plt.ylabel('Voronoi orbit bundle id')
plt.colorbar(label='log Weight')

Generate maps for each Voronoi orbit bundle. These are mass maps for mass-weighted and flux maps for light-weighted models and plot the mass/light contributed by the individual Voronoi orbital bundles at each aperture.

In [ ]:
a = dyn.analysis.Analysis(c)  # orbit bundle maps are residing in the Analysis class

In [ ]:
bundle_maps, bundle_figure = a.get_orbit_bundle_maps(pop_set=0, bundle_mapping=vor_bundle_mapping, normalize=True, sb_maps=True)

## Observed data

Let's have a look at the observed data...

In [ ]:
fig = coloring.color_maps();

## Bayesian statistical analysis

In [ ]:
# Zhu et al. 2020
# R1

stars = c.system.get_unique_triaxial_visible_component()
pops = stars.population_data[0]
age, dage, met, dmet = [pops.get_data()[i] for i in ('age', 'dage', 'met', 'dmet')]

prior_t = {'mu': np.random.normal(age.mean(), 2 * age.std(), size=len(vor_bundle_mapping)),  # (12)
           'sigma': 2 * age.std(),  # (13)
           'lower': 0,
           'upper': 20}
sample = {'n_draws': 500,
          'n_tune': 2500,
          'advi_init': 200000}

flux_data_rel = np.array([bundle_maps[a] for a in bundle_maps.columns if a != 'flux_all']).T  # shape = n_spatial_bins, n_bundle

model_t, trace_t = coloring.fit_normal(prior=prior_t,
                                       flux_data_norm=flux_data_rel,
                                       age=age,
                                       sample=sample)

# varnames = ['t_k', 'student_t_sigma', 'student_t_nu']
# pm.plot_trace(trace_t, varnames, combined=True)
pm.plot_trace(trace_t, combined=True)
# fig = plt.gcf() # to get the current figure...
# plt.close(fig)

In [ ]:
pm.model_to_graphviz(model_t)

In [ ]:
# Check how the model matches the data
age_mean_R1 = trace_t.posterior['t_k'].mean(axis=(0,1))
age_err_R1 = trace_t.posterior['t_k'].std(axis=(0,1))
model_data = {'Age R1': (age_mean_R1, age_err_R1, None, None)}
coloring.color_maps(model_data=model_data, flux_data_rel=flux_data_rel);

In [ ]:
age.mean()

In [ ]:
# Zhu et al. 2020
# R2: fit t = t_0 + p * lambda_z to the R1 model results

exclude_neg_lam_z = True  # exclude lamda_z < 0 from fitting?

idx = np.where(phase_space_binning['out'][1] >= 0)[0]
lam_z = phase_space_binning['out'][1][idx]
t = age_mean_R1[idx]
t_0, p = np.polynomial.polynomial.polyfit(lam_z, t, deg=1)
print(f'Linear fit t = t_0 + p * lambda_z: {t_0=}, {p=}')
plt.plot(lam_z, t, "o", label="R1 results")
plt.plot(lam_z, t_0 + p * lam_z, "red", label=f'Fit $t = {t_0:.3} + ({p:.3})*\lambda_z$')
plt.xlabel('$\lambda_z$')
plt.ylabel('$t$')
plt.legend()
plt.show()

In [ ]:
phase_space_binning['out'][1]

In [ ]:
# Zhu et al. 2020
# R2: map fitting

lambda_z = phase_space_binning['out'][1]  # shape = n_bundles

mu_k = np.zeros_like(lambda_z)
mu_k[lambda_z >= 0] = np.random.normal(t_0 + p * lambda_z[lambda_z >= 0], 2 * age.std() - np.abs(p) / 2)
mu_k[lambda_z < 0] = t_0

prior_t = {'mu': mu_k,  # (14)
           'sigma': 2 * age.std(),  # (15)
           'lower': 0,
           'upper': 20}
sample = {'n_draws': 500,
          'n_tune': 2500,
          'advi_init': 200000}

flux_data_rel = np.array([bundle_maps[a] for a in bundle_maps.columns if a != 'flux_all']).T  # shape = n_spatial_bins, n_bundle

model_t, trace_t = coloring.fit_normal(prior=prior_t,
                                       flux_data_norm=flux_data_rel,
                                       age=age,
                                       sample=sample)

# varnames = ['t_k', 'student_t_sigma', 'student_t_nu']
# pm.plot_trace(trace_t, varnames, combined=True)
pm.plot_trace(trace_t, combined=True)
# fig = plt.gcf() # to get the current figure...
# plt.close(fig)

In [ ]:
# Check how the model matches the data
age_mean_R2 = trace_t.posterior['t_k'].mean(axis=(0,1))
age_err_R2 = trace_t.posterior['t_k'].std(axis=(0,1))
model_data['Age R2'] = (age_mean_R2, age_err_R2, None, None)
coloring.color_maps(model_data=model_data, flux_data_rel=flux_data_rel);

In [ ]:
# Zhu et al. 2020
# Metallicity R1

# stars = c.system.get_unique_triaxial_visible_component()
# pops = stars.population_data[0]
# age, dage, met, dmet = [pops.get_data()[i] for i in ('age', 'dage', 'met', 'dmet')]

prior_z = {'mu': np.log(np.random.normal(met.mean(), met.std(), size=len(vor_bundle_mapping))),  # (17)
           'sigma': met.std(),  # (18)
           'lower': 0,
           'upper': 10}
sample = {'n_draws': 500,
          'n_tune': 2500,
          'advi_init': 200000}

# print('######## Priors: ########')
# fig = plt.figure(figsize = (10,5), layout='tight')
# fig1 = plt.subplot(2, 2, 1)
# plt.title('fit_agemet.py')
# met_start_plot = met.mean() + np.abs(np.random.normal(0, met.std(), size = len(vor_bundle_mapping)))
# sd_met_start_plot = met.std() + np.abs(np.random.normal(0, 0.01, size = len(vor_bundle_mapping)))
# plt.plot(age_mean_R1, met_start_plot, 'k.')
# plt.xlim(0, age_mean_R1.max())
# plt.ylim(-1, met_start_plot.max() + 1)
# plt.xlabel('t [Gyr]')
# plt.ylabel('$Z/Z_{sun}$')

# fig2 = plt.subplot(2, 2, 2)
# plt.title('fit_agemet.py')
# plt.plot(met_start_plot, sd_met_start_plot,'k.')
# plt.xlabel('Z')
# plt.ylabel('sigma(lnZ)')

# fig3 = plt.subplot(2, 2, 3)
# plt.title('DYNAMITE?')
# plt.plot(age_mean_R1, np.exp(prior_z['mu']), 'k.')
# plt.xlim(0, age_mean_R1.max())
# plt.ylim(-1, np.exp(prior_z['mu']).max() + 1)
# plt.xlabel('t [Gyr]')
# plt.ylabel('$Z/Z_{sun}$')

# fig2 = plt.subplot(2, 2, 4)
# plt.title('DYNAMITE?')
# plt.plot(np.exp(prior_z['mu']), prior_z['sigma']*np.ones(len(vor_bundle_mapping)),'k.')
# plt.xlabel('Z')
# plt.ylabel('sigma(lnZ)')

# plt.show()

print('######## MCMC: ########')

# flux_data_rel = np.array([bundle_maps[a] for a in bundle_maps.columns if a != 'flux_all']).T  # shape = n_spatial_bins, n_bundle

model_z, trace_z = coloring.fit_lognormal(prior=prior_z,
                                          flux_data_norm=flux_data_rel,
                                          metallicity=met,
                                          sample=sample)

# varnames = ['t_k', 'student_t_sigma', 'student_t_nu']
# pm.plot_trace(trace_t, varnames, combined=True)
pm.plot_trace(trace_z, combined=True)
# fig = plt.gcf() # to get the current figure...
# plt.close(fig)

In [ ]:
# Check how the model matches the data
met_mean_R1 = trace_z.posterior['z_k'].mean(axis=(0,1))
met_err_R1 = trace_z.posterior['z_k'].std(axis=(0,1))
model_data['Metallicity R1'] = (age_mean_R1, age_err_R1, met_mean_R1, met_err_R1)
coloring.color_maps(model_data=model_data, flux_data_rel=flux_data_rel);

## AMR

In [ ]:
fig = plt.figure(figsize = (10,5), layout='tight')
ax = plt.subplot(2, 2, 1)
plt.title('age R1 - met R1')
# met_start_plot = met.mean() + np.abs(np.random.normal(0, met.std(), size = len(vor_bundle_mapping)))
# sd_met_start_plot = met.std() + np.abs(np.random.normal(0, 0.01, size = len(vor_bundle_mapping)))
plt.plot(age_mean_R1, met_mean_R1, 'k.')
# plt.xlim(0, age_mean_R1.max())
# plt.ylim(-1, met_start_plot.max() + 1)
plt.xlabel('t [Gyr]')
plt.ylabel('$Z/Z_{sun}$')
ax = plt.subplot(2, 2, 2)
plt.title('age R2 - met R1')
plt.plot(age_mean_R2, met_mean_R1, 'k.')
plt.xlabel('t [Gyr]')
plt.ylabel('$Z/Z_{sun}$')
plt.show()

# Appendix

## Testing the orbit binning
As dithering=1, each "original" orbit bundle is just one orbit. Hence, 100% of each such orbit bundle will lie in a unique $r, \lambda_z$ input bin and the `vor_bundle_mapping` will map these "original" orbit bundles to the Voronoi orbit bundles.

To test the orbit binning, we will - for each Voronoi orbit bundle - identify the connected "original" orbit bundles and add up their weighted fractions. These should be the same whether computed (a) from the binning input data, (b) from the total weights in the Voronoi bins, and (c) from adding all "original" orbit bundles' contributions in the `vor_bundle_mapping`.

In [ ]:
for vor_bin in range(phase_space_binning['out'].shape[-1]):  # do the following for each Voronoi bin

    input_bins = np.where(phase_space_binning['map']==vor_bin)[0]
    print(f'\ninput bins mapped to {vor_bin=}: {input_bins}')

    # orbit weights in input bins:
    weight_in = [phase_space_binning['in'][2][i] for i in input_bins]  # phase_space_binning['in'].shape=(3, nr*nl), holds r, l, w for each input bin
    weight_in_total = sum(weight_in)
    print(f'{weight_in=}, {weight_in_total=}')

    vorbin_weight = phase_space_binning['out'][2][vor_bin]  # phase_space_binning['out'].shape=(3, n_bundle), holds r, l, w for each Voronoi bin
    print(f'{vorbin_weight     = }')

    vor_bundle_weight = np.sum(vor_bundle_mapping[vor_bin])  # vor_bundle_mapping.shape=(n_bundle, n_orbits), contribution of orbit-weights to Voronoi bins
    print(f'{vor_bundle_weight = }')

    print('Test succeeded? ', np.isclose(weight_in_total, vorbin_weight) and np.isclose(weight_in_total, vor_bundle_weight))

In [ ]:
stars.population_data

In [ ]:
pm.model_to_graphviz(model_z)

In [ ]:
plotter.orbit_plot(Rmax_arcs=200)